Leggere il txt cercando di non salvare le righe che comprendono solo invii.

In [ ]:
var_openTXT = open('/content/drive/My Drive/poggioni/training.txt','r').readlines()
var_cleanTXT = []
for i in range(0,len(var_openTXT)):
  if ((var_openTXT[i]) != '\n'):
    var_cleanTXT.append(var_openTXT[i])
#for i in range(0,len(var_cleanTXT)):
print(var_cleanTXT[2])

Anche tu di ritorno dal regno dell'Ade?



Creazione del csv sfruttando la struttura di questo pseudo xml  ottenendo vari dai inseriti nel file txt

In [ ]:
import csv
post = ""
topic= ""
gander = ""
top_lenpost = 0
flag = True 
lista_topic = ['ANIME','MEDICINE-AESTHETICS','AUTO-MOTO','SPORTS','SMOKE','METAL-DETECTING','CELEBRITIES','ENTERTAINMENT','TECHNOLOGY','NATURE','BIKES']
lista_age = ['0-19','20-29','30-39','40-49','50-100']
with open('training.csv','w', newline='') as f:
  fieldnames = ['post', 'topic','age','gender', 'number word']
  thewriter = csv.DictWriter(f,fieldnames=fieldnames)
  thewriter.writeheader()
  for i in range (0,len(var_cleanTXT)):
  #for i in range (0,10000):
    if '<user' in var_cleanTXT[i]:
      #salva i dati utili e poi continua
      dati_utili = var_cleanTXT[i].split(" ")

      #PARTE RIGUARDANTE IL TOPIC
      var = dati_utili[2].split('"')
      topic = var[1]
      print(dati_utili)
      if topic not in lista_topic:
        print("ERRORE NEL TOPIC")
      #---------------------------
      #PARTE RIGUARDANTE IL AGE
      var = dati_utili[3].split('"')
      age = var[1]
      if age not in lista_age:
        print("ERRORE NEL AGE")
      #---------------------------
      #PARTE RIGUARDANTE IL GANDER
      var =dati_utili[4].split('"')
      gender = var[1]
      if gender != "M" and gender != "F":
        print("ERRORE NEL GANDER")
      if gender == "M":
        gender = 0
      if gender == "F":
        gender = 1
      #----------------------------
      print("valore di i -->",i)
      flag = False
    if '<post>\n' in var_cleanTXT[i]:
      #controlla che la variabile post sia pulita in caso continua
      if (post != "" ):
        print("ERRORE")
        print("valore di i -->",i)
      flag = True
      continue
    if '</post>\n' in var_cleanTXT[i]:
      #scrivi sul csv e pulisci la variabile
      if (post[-1:]=='\n'):
        #print("ho trovato un invio finale")
        post = post[:-1]
      thewriter.writerow({'post': post, 'topic': lista_topic.index(topic) ,'age' : lista_age.index(age) ,'gender' : gender,'number word':int(len(post))})
      #thewriter.writerow({'post': post, 'topic': topic ,'age' : age ,'gender' : gender,'number word':int(len(post))})
      if top_lenpost < len(post):
        top_lenpost = len(post)
      #print("ho scritto sul csv --> " , post)
      post = ""    
      flag = False
    #siamo nel caso dei primi tre che sono risultati falsi
    #scrivi sulla variabile post il contenuto della riga 
    if flag: 
      post =  post + var_cleanTXT[i]
  

In [ ]:
print("il post più lungo è")
print(top_lenpost)

il post più lungo è
50135


Visualizzazione delle prime dieci righe del nuovo csv per rendersi conto della struttura di quest'ultimo. (google colab non apre il csv in visualizzazione perchè troppo grande)

In [ ]:
import pandas as pd
traning = pd.read_csv('/content/training.csv')
df = pd.DataFrame(traning)
print(df.head(10))

                                      post  topic  age  gender  number word
0  Anche tu di ritorno dal regno dell'Ade?      0    1       0           39
1                         Benvenuto, Uomo!      0    1       0           16
2                         Benvenuto, Uomo!      0    1       0           16
3                         Benvenuto, Uomo!      0    1       0           16
4                         Benvenuto, Uomo!      0    1       0           16
5                        Bentornato, Uomo!      0    1       0           17
6                        Benvenuta...Donna      0    1       0           17
7                                AHAHAH!!!      0    1       0            9
8                                Ciao Uomo      0    1       0            9
9                                Ciao Uomo      0    1       0            9


In [ ]:
with open('/content/training.csv','r') as in_file, open('traning_noduplicate.csv','w') as out_file:
    seen = set() # set for fast O(1) amortized lookup
    for line in in_file:
        if line in seen: continue # skip duplicate

        seen.add(line)
        out_file.write(line)

In [ ]:
traning = pd.read_csv('/content/traning_noduplicate.csv')
df = pd.DataFrame(traning)
print(df.head(20))

                                                 post  ...  number word
0             Anche tu di ritorno dal regno dell'Ade?  ...           39
1                                    Benvenuto, Uomo!  ...           16
2                                   Bentornato, Uomo!  ...           17
3                                   Benvenuta...Donna  ...           17
4                                           AHAHAH!!!  ...            9
5                                           Ciao Uomo  ...            9
6                                          Ciao donna  ...           10
7                                       Aug!!! uri...  ...           13
8                            Benvenuto, piccolo uomo.  ...           24
9                                    Ciao Auguroni...  ...           16
10  Maluska 22 Princ 24 WandeFullStar 25 Kuja 27 A...  ...          428
11  OK...spero che nessun altro abbia scelto Pavon...  ...          178
12  Grazie per l'intervento kim... davvero notevol...  ...      

In [ ]:
num_topic=[0,0,0,0,0,0,0,0,0,0,0]
num_word_topic=[0,0,0,0,0,0,0,0,0,0,0]
num_age=[0,0,0,0,0]
num_word_age=[0,0,0,0,0]


Pre processing del testo con relativa pulizia per ottenere un risultato migliore in termini di acuratezza 

In [ ]:
#continuare con la pulizia del testo e il conteggio dei smile 
import nltk
import pandas as pd
from nltk.stem import PorterStemmer #https://www.nltk.org/howto/stem.html
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.snowball import ItalianStemmer
import re

#per torliere la punteggiatura
import string
punctuations = '''!()-[]{};:'"\,<>./?@#$%^&*_...~''' 
nltk.download('punkt')
#download the stopwords
nltk.download('stopwords')
#load the italian stopwords
stops = set(stopwords.words("italian"))
stopsuk = set(stopwords.words("english"))
#rimuovo dalle stop word alcune parole per non compromettere la predizione del sesso 
stops.remove('uno')
stops.remove('una')
stops.remove('quella')
stops.remove('quello')
#cerco le emoji
!pip install demoji
import demoji 
demoji.download_codes()
numero_vuoti=[0]
numero_frasi=[0]
#-----------------------------------------------------------------------------------------------#
def apply_cleaning_function_to_list(X):
    cleaned_X = []
    for element in X:
        cleaned_X.append(clean_text(element))
    return cleaned_X

def clean_text(raw_text):
  
  string = raw_text
  
  #cancello le emoji
  #demoji.replace(string,"")
  #rimpiazzare le emoji con la loro descrizione
  demoji.replace_with_desc(string,"")
  url = Find(string)
  for i in range(0,len(url)):
    string = string.replace(url[i],'')
  #eseguo il tokeniker tramite le parole italiane
  tokens = nltk.word_tokenize(string, language='italian')
  
  #elimino le stopwords italiane e le parole più corte di 2
  text_nostopwordsit = [ w for w in tokens if not w.lower() in stops and len(w)>2]
  
  #elimino le stopword inglesi (ho notaato che alcuni post sono scritti in parte in inglese)
  text_nostopwords = [ w for w in text_nostopwordsit if not w.lower() in stopsuk ]

  # rendo tutte le lettere minuscole
  for i in range(0, len(text_nostopwords)):
    text_nostopwords[i]=text_nostopwords[i].lower()
  
  #vado a eliminare i tag ad altri account 
  for i in range(0, len(text_nostopwords)):
    string = text_nostopwords[i]
    if string[0] == '@':
      text_nostopwords[i] = ''

  #vado ad eliminare la punteggiatura
  text_nopunct = [ w for w in text_nostopwords if not w in punctuations]
  
  #vado ad eliminare i numeri presenti nel testo
  text_nonumber = [w for w in text_nopunct if  w.isalpha() ] 

  #"riatacco" le parole precedentemente separate per effettuare la pulizia 
  joined_words = ( " ".join(text_nonumber))

  if joined_words == "":
    numero_vuoti.append(0)
    #print(joined_words)
  numero_frasi.append(0)
  #ritorno la stringa da scrivere nel csv 
  return joined_words

def Find(string): 

	# findall() has been used 
	# with valid conditions for urls in string 
	regex = r"(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))"
	url = re.findall(regex,string)
	return [x[0] for x in url] 



#------------------------------------------------------------------------------------------------#

imdb = pd.read_csv('/content/traning_noduplicate.csv')


#imdb = imdb.head(15) #serve per prendere solo le righe indicate 

# Get text to clean
text_to_clean = list(imdb['post'])


for i in range (0, len(text_to_clean)):
  text_to_clean[i] = str(text_to_clean[i])
# Clean text
cleaned_text = apply_cleaning_function_to_list(text_to_clean)

# Show example
print ('Original text:',text_to_clean[15])
print ('\nCleaned text:', cleaned_text[15])



# Add cleaned data back into DataFrame
imdb['post'] = cleaned_text
imdb.to_csv('traning_clean.csv', index=False) #True per avere id prima della frase
print(len(numero_vuoti))
print(len(numero_frasi))
#togliere i duplicati

In [ ]:
for i in range(0,200):
  print(cleaned_text[i],i)

In [ ]:
import csv

with open("/content/traning_clean.csv", 'r') as f:
  data = list(csv.reader(f))
print(data[141])
with open("traning_no_vuoti.csv",'w') as f:
  writer = csv.writer(f)
  for row in data:
    if row[0]!= "":
      writer.writerow(row)

In [ ]:
file = open("/content/traning_clean.csv")
reader = csv.reader(file)
lines = len(list(reader))
print(lines)
file1 = open("/content/traning_no_vuoti.csv")
reader1 = csv.reader(file1)
lines1 = len(list(reader1))
print(lines1)

In [ ]:
#controllo frase più lunga dopo la pulizia

len_embeding = 0
for i in range(0,len(cleaned_text)):
  if len_embeding < len(cleaned_text[i]):
    len_embeding = len(cleaned_text[i])
    
print(len_embeding)